# API for SimpleFIX : Algorithmic Code Generation using QuickFIX's XML File.

The code generated below is an API for SimpleFix for FIX version 4.4.

SimpleFIX : https://simplefix.readthedocs.io/en/latest

SimpleFIX GitHub: https://github.com/da4089/simplefix

QuickFIX: http://www.quickfixengine.org/

We will create a function for each message type, with all required tags as compulsory arguments to the function, and all non-essential tags as default arguments which will be 'None' by default.

For Components in FIX, we define classes for each component inheriting from a base Component Class, each of which need to have a tag_vals attribute. All the tags and corresponding values will be stored in this when the component is initialised. These components have to be provided to the message as arguments(an instance of the corresponding component class).

```python
class Component:
    def __init__(self):
        self.tag_vals = []
```

As before, all required tags are compulsory arguments to the constructor function (when the object is created), and all non-essential tags as default arguments which will be 'None' by default.

For Groups, we use a special structure which is as follows:
```python
group_name = [[Tag_1_Name, Tag_2_Name, ... Tag_M_Name],
              [Value_Tag_1, Value_Tag_2, ... Value_Tag_M], #For group instance 1
              [Value_Tag_1, Value_Tag_2, ... Value_Tag_M], #For group instance 2
              .
              .
              .
              [Value_Tag_1, Value_Tag_2, ... Value_Tag_M]] #For group instance N
```

Thus, the number of instances of the group is 
```python
len(group_name)-1
```

For parsing a received message, we create a function parse_message(), which takes as input an instance of the SimpleFIX FixMessage, which is obtained after calling the FixParser.get_message(). This is decoded to provide names of tags instead of mere numbers, and returned in a structure as follows:

```python
msg_dict = {Tag_Name_1:Val_1,
            Tag_Name_2:Val_1,
            .
            .
            .
            Tag_Name_N:Val_N}
```

Groups are returned in the same structure as before, thus allowing convenient usage of these.

## Code Generation

### xmltodict will read the xml file and convert to a dictionary so we can expolit the structure of the XML File.

In [1]:
import xmltodict

In [2]:
#Open the file, and create the dictionary.
file = open('FIX44.xml', mode = 'rb')
data = xmltodict.parse(file)

In [3]:
groups_to_fields = {} #Will store the mappings of GroupName to Fields and Components

In [4]:
header_required_tags = []
x=data['fix']['header']
for field in x['field']:
    if field['@required'] == 'N':
        header_required_tags.append(field['@name'])
groups_to_fields[x['group']['@name']] = [i['@name'] for i in x['group']['field']]
header_required_tags.append(x['group']['@name'])

In [5]:
tag_num_to_name = [None]*957 #Will store the mapping of tag name to tag number
tag_name_to_num = {} #Will store the reverse mapping
tag_id_desc_to_num = [None]*957 #Will store the mappings of values allowed if type is enum

In [6]:
for x in data['fix']['fields']['field']:
    tag_num_to_name[int(x['@number'])] = x['@name']
    tag_name_to_num[x['@name']] = int(x['@number'])
    if 'value' in x.keys():
        tag_id_desc_to_num.insert(int(x['@number']),{k['@description']:k['@enum'] for k in x['value']})

In [7]:
components_names = [i['@name'] for i in data['fix']['components']['component']] #Will store the names of the components

In [8]:
component_class_templates = [['class Component:', '\tdef __init__(self):','\t\tself.tag_vals = []']] #Will store the Component Classes Code

In [9]:
for x in data['fix']['components']['component']:
    components_classes = [] #Code for current Component Class
    c_name = x['@name'] #Name of the Component
    components_classes.append(f'class {c_name}(Component):')
    reqd = [] #Required Tags
    not_reqd = [] #Non-Required Tags
    
    if 'field' in x.keys():
        fields = x['field']
        if type(fields)==list:
            for arg in fields:
                if arg['@required'] == 'Y':
                    reqd.append(f"{arg['@name']}")
                else:
                    not_reqd.append(f"{arg['@name']}=None")
        else:
            if fields['@required'] == 'Y':
                reqd.append(f"{fields['@name']}")
            else:
                not_reqd.append(f"{fields['@name']}=None")
                                
    if 'component' in x.keys():
        components = x['component']
        if type(components)==list:
            for arg in components:
                if arg['@required'] == 'Y':
                    reqd.append(f"{arg['@name']}")
                else:
                    not_reqd.append(f"{arg['@name']}=None")
        else:
            if components['@required'] == 'Y':
                reqd.append(f"{components['@name']}")
            else:
                not_reqd.append(f"{components['@name']}=None")
                                
    if 'group' in x.keys():
        groups_m = x['group']
        if type(groups_m)==list:
            for arg in groups_m:
                if arg['@required'] == 'Y':
                    reqd.append(f"{arg['@name']}")
                else:
                    not_reqd.append(f"{arg['@name']}=None")
                #Groups contain other fields, so that information must be stored.
                if 'field' in arg.keys():
                    if type(groups_m['field']) == list:
                        groups_to_fields[arg['@name']] = [i['@name'] for i in arg['field']]
                    else:
                        groups_to_fields[groups_m['@name']] = [groups_m['field']['@name']]
                #Groups contain other components, so that information must be stored.
                if 'component' in arg.keys():
                    if type(groups_m['component']) == list:
                        groups_to_fields[arg['@name']] += [i['@name'] for i in arg['component']]
                    else:
                        groups_to_fields[groups_m['@name']] += [groups_m['component']['@name']]
        else:
            if groups_m['@required'] == 'Y':
                reqd.append(f"{groups_m['@name']}")
            else:
                not_reqd.append(f"{groups_m['@name']}=None")
            
            if 'field' in groups_m.keys():
                if type(groups_m['field'])==list:
                    groups_to_fields[groups_m['@name']] = [i['@name'] for i in groups_m['field']]
                else:
                    groups_to_fields[groups_m['@name']] = [groups_m['field']['@name']]
            if 'component' in groups_m.keys():
                if type(groups_m['component'])==list:
                    groups_to_fields[groups_m['@name']] = [i['@name'] for i in groups_m['component']]
                else:
                    groups_to_fields[groups_m['@name']] = [groups_m['component']['@name']]
                                
    components_classes.append(f"\tdef __init__(self, {(','.join(reqd+not_reqd))}):")
    components_classes.append('\t\tself.tag_vals = []')
                                
    for tag in reqd: #Iterate through each compulsory tag
        if tag not in components_names: 
            if tag not in groups_to_fields.keys():
                # It's a tag
                components_classes.append(f'\t\tself.tag_vals.append(({tag_name_to_num[tag]},{tag}))')
            else:
                # It's a group
                components_classes.append(f'\t\tself.tag_vals.append(({tag_name_to_num[tag]},len({tag})-1))')
                components_classes.append(f'\t\tfor i in range(1,len({tag})):')
                components_classes.append(f'\t\t\tfor idx,attr in enumerate({tag}[0]):')
                components_classes.append(f'\t\t\t\tif attr in components_names:')
                components_classes.append(f'\t\t\t\t\tself.tag_vals+=attr.tag_vals')
                components_classes.append(f'\t\t\t\telse:')
                components_classes.append(f'\t\t\t\t\tself.tag_vals.append(tag_name_to_num[attr],{tag}[i][idx])')
        else:
            #It's a component
            components_classes.append(f'\t\tself.tag_vals+={tag}.tag_vals')
    for tag in not_reqd:
        tag = tag[:-5] #To remove the "=None" at the end of non-essential tags that we added previously
        components_classes.append(f'\t\tif {tag} != None:')
                                
        if tag not in components_names:
            if tag not in groups_to_fields.keys():
                #It's a tag
                components_classes.append(f'\t\t\tself.tag_vals.append(({tag_name_to_num[tag]},{tag}))')
            else:
                #It's a group
                components_classes.append(f'\t\t\tself.tag_vals.append(({tag_name_to_num[tag]},len({tag})-1))')
                components_classes.append(f'\t\t\tfor i in range(1,len({tag})):')
                components_classes.append(f'\t\t\t\tfor idx,attr in enumerate({tag}[0]):')
                components_classes.append(f'\t\t\t\t\tif attr in components_names:')
                components_classes.append(f'\t\t\t\t\t\tself.tag_vals+=attr.tag_vals')
                components_classes.append(f'\t\t\t\t\telse:')
                components_classes.append(f'\t\t\t\t\t\tself.tag_vals.append(tag_name_to_num[attr],{tag}[i][idx])')
        else:
            #It's a Component
             components_classes.append(f'\t\t\tself.tag_vals+={tag}.tag_vals')
    component_class_templates.append(components_classes)

In [11]:
final_templates_messages = [] #Will store the code for the messages functions.

In [12]:
for msg in data['fix']['messages']['message']:
    final_templates = [] #Code of current message
    name = msg['@name'] #Name of message
    msg_type = msg['@msgtype']
    reqd = []
    not_reqd = []
    
    #For all field attributes
    if 'field' in msg.keys():
        fields = msg['field']
        if type(fields)==list:
            for arg in fields:
                if arg['@required'] == 'Y':
                    reqd.append(f"{arg['@name']}")
                else:
                    not_reqd.append(f"{arg['@name']}=None")
        else:
            if fields['@required'] == 'Y':
                reqd.append(f"{fields['@name']}")
            else:
                not_reqd.append(f"{fields['@name']}=None")
                                
    #For all component type attributes
    if 'component' in msg.keys():
        components = msg['component']
        if type(components)==list:
            for arg in components:
                if arg['@required'] == 'Y':
                    reqd.append(f"{arg['@name']}")
                else:
                    not_reqd.append(f"{arg['@name']}=None")
        else:
            if components['@required'] == 'Y':
                reqd.append(f"{components['@name']}")
            else:
                not_reqd.append(f"{components['@name']}=None")
                                
    if 'group' in msg.keys():
        groups_m = msg['group']
        if type(groups_m)==list:
            for arg in groups_m:
                if arg['@required'] == 'Y':
                    reqd.append(f"{arg['@name']}")
                else:
                    not_reqd.append(f"{arg['@name']}=None")
                if 'field' in arg.keys():
                    if type(groups_m['field']) == list:
                        groups_to_fields[arg['@name']] = [i['@name'] for i in arg['field']]
                    else:
                        groups_to_fields[groups_m['@name']] = [groups_m['field']['@name']]
                if 'component' in arg.keys():
                    if type(groups_m['component']) == list:
                        groups_to_fields[arg['@name']] += [i['@name'] for i in arg['component']]
                    else:
                        groups_to_fields[groups_m['@name']] += [groups_m['component']['@name']]
        else:
            if groups_m['@required'] == 'Y':
                reqd.append(f"{groups_m['@name']}")
            else:
                not_reqd.append(f"{groups_m['@name']}=None")
            if 'field' in groups_m.keys():
                if type(groups_m['field'])==list:
                    groups_to_fields[groups_m['@name']] = [i['@name'] for i in groups_m['field']]
                else:
                    groups_to_fields[groups_m['@name']] = [groups_m['field']['@name']]
            if 'component' in groups_m.keys():
                if type(groups_m['component'])==list:
                    groups_to_fields[groups_m['@name']] = [i['@name'] for i in groups_m['component']]
                else:
                    groups_to_fields[groups_m['@name']] = [groups_m['component']['@name']]
    
    final_templates.append('def '+name+'(MsgSeqNum,TargetCompID,'+','.join(reqd+not_reqd)+'):')
    final_templates.append('\tmsg=simplefix.FixMessage()')
    final_templates.append('\tmsg.append_pair(8,"FIX.4.4",header=True)')
    final_templates.append(f'\tmsg.append_pair(35,{msg_type},header=True)')
    final_templates.append('\tmsg.append_pair(49,SENDER_COMP_ID,header=True)')
    final_templates.append('\tmsg.append_pair(34,MsgSeqNum,header=True)')
    final_templates.append('\tmsg.append_pair(56,TargetCompID,header=True)')
    for tag in reqd:
        if tag not in components_names:
            if tag not in groups_to_fields.keys():
                final_templates.append(f'\tmsg.append_pair({tag_name_to_num[tag]},{tag})')
            else:
                final_templates.append(f'\tself.tag_vals.append(({tag_name_to_num[tag]},len({tag})-1))')
                final_templates.append(f'\tfor i in range(1,len({tag})):')
                final_templates.append(f'\t\tfor idx,attr in enumerate({tag}[0]):')
                final_templates.append(f'\t\t\tif attr in components_names:')
                final_templates.append(f'\t\t\t\tfor tv in attr.tag_vals:')
                final_templates.append(f'\t\t\t\t\tmsg.append_pair(tv[0],tv[1])')
                final_templates.append(f'\t\t\telse:')
                final_templates.append(f'\t\t\t\tmsg.append_pair(tag_name_to_num[attr],{tag}[i][idx])')
        else:
            final_templates.append(f'\tfor tv in {tag}.tag_vals:')
            final_templates.append(f'\t\tmsg.append_pair(tv[0],tv[1])')
    for tag in not_reqd:
        tag = tag[:-5]
        final_templates.append(f'\tif {tag} != None:')
        if tag not in components_names:
            if tag not in groups_to_fields.keys():
                final_templates.append(f'\t\tmsg.append_pair({tag_name_to_num[tag]},{tag})')
            else:
                final_templates.append(f'\t\tself.tag_vals.append(({tag_name_to_num[tag]},len({tag})-1))')
                final_templates.append(f'\t\tfor i in range(1,len({tag})):')
                final_templates.append(f'\t\t\tfor idx,attr in enumerate({tag}[0]):')
                final_templates.append(f'\t\t\t\tif attr in components_names:')
                final_templates.append(f'\t\t\t\t\tfor tv in attr.tag_vals:')
                final_templates.append(f'\t\t\t\t\t\tmsg.append_pair(tv[0],tv[1])')
                final_templates.append(f'\t\t\t\telse:')
                final_templates.append(f'\t\t\t\t\tmsg.append_pair(tag_name_to_num[attr],{tag}[i][idx])')
        else:
            final_templates.append(f'\t\tfor tv in {tag}.tag_vals:')
            final_templates.append(f'\t\t\tmsg.append_pair(tv[0],tv[1])')
    final_templates.append(f'\treturn msg.encode()')
    final_templates_messages.append(final_templates)

In [14]:
my_file = open('SimpleFIX_API_1.py','w')

In [15]:
my_file.writelines('import simplefix\n\n')
my_file.writelines('SENDER_COMP_ID = "COEP_Credit_Suisse"\n\n')
my_file.writelines(f"""components_names=[{",".join([f'"{c}"' for c in components_names])}]\n\n""")
my_file.writelines('tag_name_to_num={'+",".join([f'"{k}":{tag_name_to_num[k]}' for k in tag_name_to_num.keys()])+'}\n\n')
for f in component_class_templates:
    for j in f:
        my_file.writelines(j+'\n')
    my_file.writelines('\n')
for f in final_templates_messages:
    for j in f:
        my_file.writelines(j+'\n')
    my_file.writelines('\n')

In [16]:
my_file.close()

### Now, we generate the message parsing functions. For this we extract all tags possible in each message(including components and groups).

In [17]:
def group_tags(group): #This will return all tags in a group (including components unwrapping)
    tag_list = []
    if 'field' in group.keys():
        fields = group['field']
        if type(fields)==list:
            tag_list+=[tag_name_to_num[i['@name']] for i in fields]
        else:
            tag_list.append(tag_name_to_num[fields['@name']])
    if 'component' in group.keys():
        components = group['component']
        if type(components)==list:
            for comp in components:
                tag_list.append([comp['@name'],component_tags(comp['@name'])])
        else:
            tag_list.append([components['@name'],component_tags(components['@name'])])
    if 'group' in group.keys():
        raise ValueError()  #Since assumption of no direct recursion of groups i.e. a group within a group
    return tag_list

def component_tags(component_name): #This will return all tags in a component (including components, groups unwrapping)
    tag_list = []
    for i in data['fix']['components']['component']:
        if i['@name'] == component_name:
            component = i
            break
    if 'field' in component.keys():
        fields = component['field']
        if type(fields)==list:
            tag_list+=[tag_name_to_num[i['@name']] for i in fields]
        else:
            tag_list.append(tag_name_to_num[fields['@name']])
    if 'component' in component.keys():
        components = component['component']
        if type(components)==list:
            for comp in components:
                tag_list.append([comp['@name'],component_tags(comp['@name'])])
        else:
            tag_list.append([components['@name'],component_tags(components['@name'])])
    if 'group' in component.keys():
        groups = component['group']
        if type(groups) == list:
            for group in groups:
                tag_list.append([tag_name_to_num[group['@name']],group_tags(group)])
        else:
            tag_list.append([tag_name_to_num[groups['@name']],group_tags(groups)])
    return tag_list
                
final_tags = {} #This will store the tags corresponding to a message

for msg in data['fix']['messages']['message']:
    msg_idx = msg['@msgtype']
    tag_list = []
    if 'field' in msg.keys():
        fields = msg['field']
        if type(fields)==list:
            tag_list+=[tag_name_to_num[i['@name']] for i in fields]
        else:
            tag_list.append(tag_name_to_num[fields['@name']])
    if 'component' in msg.keys():
        components = msg['component']
        if type(components)==list:
            for comp in components:
                tag_list.append([comp['@name'],component_tags(comp['@name'])])
        else:
            tag_list.append([components['@name'],component_tags(components['@name'])])
    if 'group' in msg.keys():
        groups = msg['group']
        if type(groups) == list:
            for group in groups:
                tag_list.append([tag_name_to_num[group['@name']],group_tags(group)])
        else:
            tag_list.append([tag_name_to_num[groups['@name']],group_tags(groups)])
    final_tags[msg_idx] = tag_list

In [19]:
header_tags = []
x=data['fix']['header']
for field in x['field']:
    header_tags.append(tag_name_to_num[field['@name']])
header_tags.append([tag_name_to_num[x['group']['@name']],group_tags(x['group'])])

In [21]:
def get_msg_name(key_num): #Gets the name of the message from it's message type.
    for i in data['fix']['messages']['message']:
        if i['@msgtype'] == str(key_num):
            return i['@name']

In [22]:
def parse_component(component,num_tabs): #Generation of code when inside a component.
    for c_id in component[1]:
        if type(c_id) != list: #Field
            parse_messages_code.append('\t'*num_tabs+f'if {c_id} in msg:')
            parse_messages_code.append('\t'*num_tabs+f'\tmsg_dict["{tag_num_to_name[c_id]}"] = str(msg.get({c_id}),"utf-8")')
        elif type(c_id[0]) != str: #Group
            parse_messages_code.append('\t'*num_tabs+f'if {c_id[0]} in msg:')
            parse_messages_code.append('\t'*num_tabs+f'\tmsg_dict["{tag_num_to_name[c_id[0]]}"] = [[]]')
            parse_messages_code.append('\t'*num_tabs+f'\tfor i in range(int(msg.get({c_id[0]}))):')
            parse_messages_code.append('\t'*num_tabs+f'\t\t\tmsg_dict["{tag_num_to_name[c_id[0]]}"].append([])')
            parse_group(tag_num_to_name[c_id[0]],c_id[1],num_tabs+1)
        else: #Component
            parse_component(c_id, num_tabs)
            
def parse_group(tag_name, group_tags, num_tabs): #Generation of code when inside a group.
    for g_id in group_tags:
        if type(g_id) != list: #Field
            parse_messages_code.append('\t'*num_tabs+f'if {g_id} in msg:')
            parse_messages_code.append('\t'*num_tabs+f'\tmsg_dict["{tag_name}"][0].append("{tag_num_to_name[g_id]}")')
            parse_messages_code.append('\t'*num_tabs+f'\tfor i in range(int(msg.get({tag_name_to_num[tag_name]}))):')
            parse_messages_code.append('\t'*num_tabs+f'\t\tmsg_dict["{tag_name}"][i+1].append(str(msg.get({g_id},i+1),"utf-8"))')
        elif type(g_id[0]) != str: #Group
            parse_messages_code.append('\t'*num_tabs+f'if {tag_name_to_num[g_id[0]]} in msg:')
            parse_messages_code.append('\t'*num_tabs+f'\tmsg_dict["{tag_num_to_name[g_id[0]]}"] = [[]]')
            parse_messages_code.append('\t'*num_tabs+f'\tfor i in range(int(msg.get({g_id[0]}))):')
            parse_messages_code.append('\t'*num_tabs+f'\t\t\tmsg_dict["{tag_num_to_name[g_id[0]]}"].append([])')
            parse_group(g_id[0],g_id[1],num_tabs+1)
        else: #Component
            parse_component(g_id, num_tabs)

parse_messages_code = ['def parse_message(msg):'] #Will store the final parse_message() code
parse_messages_code.append('\tmsg_dict = {}')
parse_messages_code.append('\tmsgtype = msg.get(35)')
for h_id in header_tags:
    if type(h_id) != list:
        parse_messages_code.append(f'\tif {h_id} in msg:')
        parse_messages_code.append(f'\t\tmsg_dict["{tag_num_to_name[h_id]}"] = str(msg.get({h_id}),"utf-8")')
    elif type(h_id[0]) != str:
        parse_messages_code.append(f'\tif {h_id[0]} in msg:')
        parse_messages_code.append(f'\t\tmsg_dict["{tag_num_to_name[h_id[0]]}"] = [[]]')
        parse_messages_code.append(f'\t\tfor i in range(int(msg.get({h_id[0]}))):')
        parse_messages_code.append(f'\t\t\tmsg_dict["{tag_num_to_name[h_id[0]]}"].append([])')
        parse_group(tag_num_to_name[h_id[0]],h_id[1],2)
    
count = 0
for key in final_tags.keys():
    if count == 0:
        parse_messages_code.append(f'\tif str(msgtype,"utf-8")=="{key}":')
    elif count != 92:
        parse_messages_code.append(f'\telif str(msgtype,"utf-8")=="{key}":')
    else:
        parse_messages_code.append(f'\telse:')
    parse_messages_code.append(f'\t\tmsg_dict["{tag_num_to_name[35]}"] = "{get_msg_name(key)}"')
    for tag in final_tags[key]:
        if type(tag) != list:
            parse_messages_code.append(f'\t\tif {tag} in msg:')
            parse_messages_code.append(f'\t\t\tmsg_dict["{tag_num_to_name[tag]}"] = str(msg.get({tag}),"utf-8")')
        elif type(tag[0]) != str:
            parse_messages_code.append(f'\t\tif {tag[0]} in msg:')
            parse_messages_code.append(f'\t\t\tmsg_dict["{tag_num_to_name[tag[0]]}"] = [[]]')
            parse_messages_code.append(f'\t\t\tfor i in range(int(msg.get({tag[0]}))):')
            parse_messages_code.append(f'\t\t\t\tmsg_dict["{tag_num_to_name[tag[0]]}"].append([])')
            parse_group(tag_num_to_name[tag[0]],tag[1],3)
        else:
            parse_component(tag,2)
    count+=1
parse_messages_code.append('\treturn msg_dict')

In [23]:
my_file_parse = open('SimpleFIX_API_2.py','w')
for i in parse_messages_code:
    my_file_parse.writelines(i+'\n')
my_file_parse.close()